In [4]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import ncxt_sxtcnn 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Loaders
The loader acts as the link between the annotated SXT data and the task in hand. The loader transforms both the LAC (e.g., normalization) of the image and the annotated mask (material values) to a common format and wraps everything into an iterable class.

Loaders can be extended to include more that one channel fo input data.

In [6]:
from ncxt_sxtcnn.sxtcnn.dataloaders import NCXTMockLoader
loader = NCXTMockLoader(shape=(32, 32, 32), labels=3, length=3)

print("Loader is an iterable class of samples.\nEach sample contains:\n")
item = loader[0]
for k,v in item.items():
    if isinstance(item[k], np.ndarray):
        print(k, v.shape)
    else:
        print(k,v)

Loader is an iterable class of samples.
Each sample contains:

input (1, 32, 32, 32)
target (32, 32, 32)
key {'material_0': 0, 'material_1': 1, 'material_2': 2}


NCXTDBLoader feeds of an argument database, so that thte __getitem__ loads the actual data
``` python
def __getitem__(self, index):
        record = self.db[index]
        lac = read_mrc(record["data"])
        label = read_mrc(record["annotation"])
```

## Processor
Processors act as the intermediate dataflow from the whole samples into processable blocks.

to conform to a common standard, derive class from the ABC
DataProcessor, which contains three abstarc methods:
 - forward(self,data) -- from data to network intput 
 - backward(self, data) -- from network output to processed volume
 - init_data(*args)  -- Because of memory limitations, we initialize the train and test data to a temporary working directory. Init_data handles the serialization of this data.


In [7]:
processor = ncxt_sxtcnn.datainitializers.RandomBlockProcessor(
    block_shape=(8,8,8), 
    binning=2,  
    sampling=1.0)




## Model
The models are derived from torch.nn.Module

In [10]:
model = ncxt_sxtcnn.sxtcnn.models.UNet3D(3)

## Wrap it up!

In [14]:
params = {
    "name": f"example",
    "device": "cpu",
    "working_directory": '__tempwd__/',
}
seg = ncxt_sxtcnn.sxtcnn.sxt_cnn_wrapper.SXT_CNN_WRAPPER(
    loader, model, processor, params=params)
